In [1]:
# -*- coding: utf-8

In [2]:
# import torch
# # LLamaIndex 使用 PyTorch 进行向量计算
# # 清理GPU 资源
# torch.cuda.empty_cache()

# # 垃圾回收
# import gc
# gc.collect()

 # --can not run in notebook, run it in cmdline
# ! nvidia-smi --gpu-reset 

In [3]:
# %pip install nvidia-ml-py3
import pynvml
# 初始化 NVML 库
pynvml.nvmlInit()

# 获取 GPU 数量
num_gpus = pynvml.nvmlDeviceGetCount()
print("GPU 数量：", num_gpus)

# 遍历每个 GPU，获取其资源信息
for i in range(num_gpus):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    gpu_name = pynvml.nvmlDeviceGetName(handle)
    gpu_memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU {i}: {gpu_name.decode()}，内存使用情况：{gpu_memory_info.used / 1024 / 1024} MB / {gpu_memory_info.total / 1024 / 1024} MB")

# 关闭 NVML 库
pynvml.nvmlShutdown()

import psutil
# 获取当前进程的内存信息
process = psutil.Process()
mem_info = process.memory_info()
print("当前系统内存使用信息",mem_info)
print("物理内存中实际驻留的大小", mem_info.rss/(1024*1024*1024), "GB")
print("虚拟内存的大小，包括实际使用的物理内存和交换空间", mem_info.vms/(1024*1024*1024), "GB")
print("共享内存的大小，被多个进程共享的内存量", mem_info.shared/(1024*1024*1024), "GB")
print("可执行程序的代码段大小", mem_info.text/(1024*1024*1024), "GB")
print("程序的数据段大小", mem_info.data/(1024*1024*1024), "GB")

GPU 数量： 2
GPU 0: Tesla V100-PCIE-16GB，内存使用情况：12664.5625 MB / 16384.0 MB
GPU 1: Tesla V100-PCIE-16GB，内存使用情况：4732.5625 MB / 16384.0 MB
当前系统内存使用信息 pmem(rss=77864960, vms=720195584, shared=16674816, text=2818048, lib=0, data=148176896, dirty=0)
物理内存中实际驻留的大小 0.07251739501953125 GB
虚拟内存的大小，包括实际使用的物理内存和交换空间 0.6707344055175781 GB
共享内存的大小，被多个进程共享的内存量 0.015529632568359375 GB
可执行程序的代码段大小 0.00262451171875 GB
程序的数据段大小 0.13800048828125 GB


In [4]:


# %pip install llama-index-llms-ollama
# !pip install llama-index
# %pip install llama-index-embeddings-ollama
# %pip install docx2txt
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-embeddings-instructor

from llama_index.core import Settings

# load the ollama
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from BCEmbedding.tools.llama_index import BCERerank
from llama_index.core.node_parser import SentenceSplitter

from llama_index.core import VectorStoreIndex

# model set
llama2_7b = "llama2"
llama2_13b = "llama2:13b"
llama3_8b = "llama3"
llama3_70b = "llama3:70b"


# embedding_type = "llama3"
embedding_type = "bce"
rebuild = False

if embedding_type == "llama3":
    base_name = "ma35_rag_base"
    embedding_model = OllamaEmbedding(model_name=llama3_8b,ollama_additional_kwargs={"mirostat": 0}) #base_url="http://localhost:11434"
elif embedding_type == "bce":
    base_name = "ma35_rag_base_bce"
    embed_args = {'model_name': 'maidalun1020/bce-embedding-base_v1', 'max_length': 512, 'embed_batch_size': 256, 'device': 'cuda'}
    embedding_model = HuggingFaceEmbedding(**embed_args)
else:
    print("embedding model not correct\n")
    exit(-1)

# connect with the ollama server
llm_llama = Ollama(model=llama3_8b, request_timeout=600, temperature=0.1, device='cuda') #base_url = 'http://localhost:11434',
# llm_llama2 = Ollama(model="llama2:13b", request_timeout=600, temperature=0.1) #base_url = 'http://localhost:11434',

reranker_args = {'model': 'maidalun1020/bce-reranker-base_v1', 'top_n': 5, 'device': 'cuda'}
reranker_model = BCERerank(**reranker_args)

Settings.llm = llm_llama
Settings.embed_model = embedding_model
Settings.node_parser = SentenceSplitter(chunk_size=500, chunk_overlap=20)
# Settings.num_output = 512
# Settings.context_window = 1024

/home/xilinx/Documents/llamaindex_ma35_rag/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
05/04/2024 12:08:36 - [INFO] -sentence_transformers.SentenceTransformer->>>    Load pretrained SentenceTransformer: maidalun1020/bce-embedding-base_v1
/home/xilinx/Documents/llamaindex_ma35_rag/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
05/04/2024 12:08:41 - [INFO] -sentence_transformers.SentenceTransformer->>>    2 prompts are loaded, with the keys: ['query', 'text']
05/04/2024 12:08:43 - [INFO] -BCEmbedding.models.RerankerModel->>>    Loading from `maidalun1020/bce-reranker-base_v1`.
05/04/2024 12:08:44 - [INFO] 

In [5]:
# create a vector storage
# %pip install llama-index-vector-stores-chroma
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# initialize client, setting path to save data
chroma_client = chromadb.PersistentClient()

05/04/2024 12:08:44 - [INFO] -chromadb.telemetry.product.posthog->>>    Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [6]:
from IPython.display import Markdown, display

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [7]:

if rebuild :
    # %pip install llama-index-readers-web

    from llama_index.readers.web import SimpleWebPageReader

    documents = SimpleWebPageReader(html_to_text=True).load_data(
        [
        "https://amd.github.io/ama-sdk/v1.1.2/index.html",
        "https://amd.github.io/ama-sdk/v1.1.2/getting_started_on_prem.html",
        "https://amd.github.io/ama-sdk/v1.1.2/virtualization.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/ffmpeg/tutorials.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/ffmpeg/quality_analysis.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/ffmpeg/filters.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/gstreamer/tutorials.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/gstreamer/filters.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/gstreamer/xcompositor.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/gstreamer/xabrladder.html",
        "https://amd.github.io/ama-sdk/v1.1.2/examples/xma/xma_apps.html",
        "https://amd.github.io/ama-sdk/v1.1.2/specs_and_features.html",
        "https://amd.github.io/ama-sdk/v1.1.2/package_feed.html",
        "https://amd.github.io/ama-sdk/v1.1.2/using_ffmpeg.html",
        "https://amd.github.io/ama-sdk/v1.1.2/using_gstreamer.html",
        "https://amd.github.io/ama-sdk/v1.1.2/unified_logging.html",
        "https://amd.github.io/ama-sdk/v1.1.2/tuning_video_quality.html",
        "https://amd.github.io/ama-sdk/v1.1.2/tuning_pipeline_latency.html",
        "https://amd.github.io/ama-sdk/v1.1.2/managing_compute_resources.html",
        "https://amd.github.io/ama-sdk/v1.1.2/c_apis.html",
        "https://amd.github.io/ama-sdk/v1.1.2/card_management.html",
        "https://amd.github.io/ama-sdk/v1.1.2/encoder_comp_matrix.html",
        "https://ffmpeg.org/ffmpeg.html",
        "https://ffmpeg.org/ffmpeg-resampler.html",
        "https://ffmpeg.org/ffmpeg-devices.html",
        "https://ffmpeg.org/ffmpeg-all.html",
        "https://trac.ffmpeg.org/wiki/Encode/H.264",
        "https://trac.ffmpeg.org/wiki/Encode/H.265",
        "https://trac.ffmpeg.org/wiki/Encode/AV1",
        "https://trac.ffmpeg.org/wiki/Scaling",
        "https://trac.ffmpeg.org/wiki/Null",
        "https://trac.ffmpeg.org/wiki/FilteringGuide",
        ]
        
    )

    collection_name = base_name
    collection = chroma_client.list_collections()
    if collection_name in collection:
        chroma_client.delete_collection(collection_name)
        chroma_client.clear_system_cache()
    chroma_collection = chroma_client.get_or_create_collection(name=collection_name)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(docstore=documents, vector_store=vector_store)

    # 这个nodes 有什么用处
    from llama_index.core.node_parser import SimpleNodeParser 
    # Initialize the parser 
    parser = SimpleNodeParser.from_defaults(chunk_size=500, chunk_overlap=20) 
    # Parse documents into nodes 
    nodes = parser.get_nodes_from_documents(documents)
    # print(nodes[0])
    len(nodes)

    # %pip install ipywidgets
    # index = VectorStoreIndex.from_documents(documents,storage_context=storage_context,show_progress=True)
    index = VectorStoreIndex(nodes,embed_model=embedding_model,storage_context=storage_context,show_progress=True)

    # # documents

In [8]:
# from bs4 import BeautifulSoup

# # 读取 HTML 文件
# with open("local_html/FFMPEG command line arguments - VideoDC - Xilinx Enterprise Wiki.html", "r", encoding="utf-8") as file:
#     html_content = file.read()

# # 使用 Beautiful Soup 解析 HTML
# soup = BeautifulSoup(html_content, "html.parser")

# # 提取文档内容
# document_text = soup.get_text()

# # 打印文档内容
# print(document_text)


In [9]:
# load index from stored vectors
collection_name = base_name
collection = chroma_client.list_collections()
chroma_collection = chroma_client.get_or_create_collection(name=collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(
    vector_store, embed_model=embedding_model,storage_context=storage_context
)

In [10]:
questions = [
   """explain following ffmpeg command "ffmpeg -hwaccel ama -f rawvideo -s 1920x1080 -framerate 24 -i cut1_1080p.nv12 -vf 'hwupload' -c:v av1_ama -b:v 5M -f mp4 -y 1.av1_1080p_1.mp4" """,

   """explain following ffmpeg command
   ffmpeg -y -hwaccel ama \
      -c:v h264_ama  -out_fmt nv12 -i <INPUT>  \
      -filter_complex "scaler_ama=outputs=4:out_res=(1920x1080|full|nv12)(1280x720|full|nv12)(720x480|full|nv12)(360x240|full|nv12) [a][b][c][d]; \
                     [a]hwdownload,format=nv12[a1];[b]hwdownload,format=nv12[b1];[c]hwdownload,format=nv12[c1];[d]hwdownload,format=nv12[d1]" \
      -map '[a1]' -f rawvideo -pix_fmt nv12 -y /tmp/scale_1080p.yuv \
      -map '[b1]' -f rawvideo -pix_fmt nv12 -y /tmp/scale_720p.yuv  \
      -map '[c1]' -f rawvideo -pix_fmt nv12 -y /tmp/scale_480p.yuv \
      -map '[d1]' -f rawvideo -pix_fmt nv12 -y /tmp/scale_240p.yuv""",

    
    """Using ffmpeg Decoder a clip that is already encoded in H.264, and will decode the file into a RAW format and save it to disk.""",
    """Using ffmpeg encode a RAW 1080p60 clip in YUV420 format. Pass the clip to the MA35D encoder to produce an AV1 encoded MP4 output with a target bitrate of 5Mbps and saves it to disk. please do not use -re option""",
    """ Using ffmpeg do the Bit Conversion, To encode YUV 4:2:2 10 bit pixel format to YUV 4:2:0 8 bit format' """,
    """ Using ffmpeg decodes an existing H.264 file and then scales it into 1080p/720p/480p/240p four resolutions and save the RAW outputs to disk under""",
    """ Using ffmpeg one cmd line, decodes an existing H.264 file and then using scaler_ama scales it into 1080p/720p/480p/240p four resolutions and save the RAW outputs to disk under""",

    """How to tuning Latency of MA35D ama codec video transcode,  for example enable ultra low latency for ama_av1 Encoding?""",
    
    "一个ma35D AV1 codec transcode 的处理能力是多少 ",

   # the question come from forum
    """Is max_bitrate implemented in ama sdk ffmpeg, and is it correct that the max bitrate parameter in AMA is irrelevant to VBR?""",

]

In [11]:
# retriever = index.as_retriever()
# relevant_docs = retriever.retrieve("what is the max transcode rate for 1080p30 stream")
# relevant_docs

# """
# response_mode

#     REFINE = "refine"
#     COMPACT = "compact"
#     SIMPLE_SUMMARIZE = "simple_summarize"
#     TREE_SUMMARIZE = "tree_summarize"
#     GENERATION = "generation"
#     NO_TEXT = "no_text"

# """

# ResponseMode为tree_summarize时，LLM会对每一段文本进行最大长度的分割，并进行连续的读取和询问。这种模式的优点是可以保证对文本的完整理解和回答，但如果没有正确处理分割段落的情况，可能会导致错误的生成结果
# ResponseMode为generation时，生成的回答不依赖于文档的内容，只基于提供的问题进行生成。这种模式适用于纯粹的问题回
# ResponseMode为no_text时，生成的回答中不包含任何内容，仅作为占位符使用
# ResponseMode为simple_summarize时，LLM会截取每段文本的相关句子（通常是第一句），并进行提炼生成回答。这种模式适用于对结果要求不高的场景。
# ResponseMode为refine时，如果只有一个文本块（text_chunk），则会正常生成回答。但如果存在多个文本块，则会以类似轮询的方式迭代生成回答。这种模式可以对多个文本块进行迭代式的回答生成，逐步完善回答内容。
# ResponseMode为compact时，生成的回答会将多个文本块（text_chunk）压缩到设定的最大长度，并生成一次回答。然后，根据后续内容对以往的答案进行改进和完善（即进行多次迭代）


In [12]:
from llama_index.core import PromptTemplate
query_engine = index.as_query_engine(response_mode='simple_summarize', similary_threshold=0.1, similarity_top_k=5)

template = (
    "You are hellpful, respectful and honest video transcode assistant and very faimilay with ffmpge, and expecially good at MA35D AMA(AMD multimidia accelerator) device encode/decode/transcode.\n"
    "Context information from multiple sources is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the information from multiple sources and not prior knowledge\n"
    "please read the above context information carefully. and anwer the question.\n"
    "if the question is not releate with video process, just say it is not releated with my knowledge base.\n"
    "if you don't know the answer, just say that I don't know.\n"
    "Answers need to be precise and concise.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_template = PromptTemplate(template)


query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

You are hellpful, respectful and honest video transcode assistant and very faimilay with ffmpge, and expecially good at MA35D AMA(AMD multimidia accelerator) device encode/decode/transcode.
Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge
please read the above context information carefully. and anwer the question.
if the question is not releate with video process, just say it is not releated with my knowledge base.
if you don't know the answer, just say that I don't know.
Answers need to be precise and concise.
Query: {query_str}
Answer: 


<br><br>

In [13]:

# Here no database input, so no answer
counter = 0
for question in questions:
   counter = counter + 1
   query_response = query_engine.query(question)
   print(f"Question{counter}: {question}")
   print(f"Answer:{query_response.response}")
   

   # print(" ")
   # print(f"source_nodes length:{len(query_response.source_nodes)}")
   # for i, result in enumerate(query_response.source_nodes, start=1):
   #    print(result)

   #    # print(f"Result {i}: Document ID {result['id']}, Title '{result['title']}', Similarity: {result.score}")
   #    print(f"Result {i}\n Similarity: {result.score}\n content '{result.get_content()}")

   # print()
   # # print(response.get_formatted_sources(length=10))
   # print()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

05/04/2024 12:08:54 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Question1: explain following ffmpeg command "ffmpeg -hwaccel ama -f rawvideo -s 1920x1080 -framerate 24 -i cut1_1080p.nv12 -vf 'hwupload' -c:v av1_ama -b:v 5M -f mp4 -y 1.av1_1080p_1.mp4" 
Answer:This FFmpeg command is used to encode a raw video file with the YUV 4:2:0 (NV12) format into an MP4 container using the AV1 codec.

Here's a breakdown of the command:

* `-hwaccel ama`: This flag enables the AMD MultiMedia Accelerator (AMA) hardware acceleration for the FFmpeg process.
* `-f rawvideo`: This flag specifies that the input file is in raw video format.
* `-s 19:20x10:80`: This flag sets the resolution of the input file to 1920x1080 pixels.
* `-framerate 24`: This flag sets the frame rate of the input file to 24 frames per second.
* `-i cut1_10:80p.nv12`: This flag specifies the input file, which is a raw video file with the YUV 4:2:0 (NV12) format and a resolution of 1920x1080 pixels.
* `-vf 'hwupload'`: This flag applies the `hwupload` filter to the input file. The `hwupload` fil

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

05/04/2024 12:09:05 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


...

05/04/2024 12:09:37 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Question9: 一个ma35D AV1 codec transcode 的处理能力是多少 
Answer:Based on the provided context information, it seems that the MA35D card is capable of transcoding to and from HEVC, AVC, and AV1 formats at an aggregate rate of up to two 4Kp60 for AVC and HEVC formats, and four 4Kp60 for AV1 format, per VPU.

However, I couldn't find any specific information on the processing power or throughput of the MA35D card for a single AV1 codec transcode. The provided tables seem to focus more on the video processing power of the MA35D cards in general, rather than a specific transcode scenario.

Therefore, I don't have enough information to provide an accurate answer to this question.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

05/04/2024 12:09:42 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Question10: Is max_bitrate implemented in ama sdk ffmpeg, and is it correct that the max bitrate parameter in AMA is irrelevant to VBR?
Answer:Based on the provided context information, I can answer your question as follows:

The `max_bitrate` parameter is not explicitly mentioned in the AMA SDK documentation. However, according to the FFmpeg documentation, the `-maxrate` and `-bufsize` options are used to constrain the maximum bitrate used by the encoder.

Regarding the relevance of the `max_bitrate` parameter to VBR (Variable Bit Rate), it seems that the `max_bitrate` is indeed relevant to VBR. According to the FFmpeg documentation, when using VBR, the encoder will adjust the bitrate per-frame to achieve a certain average bitrate. The `-maxrate` and `-bufsize` options are used to constrain this average bitrate.

In summary:

* The `max_bitrate` parameter is not explicitly mentioned in the AMA SDK documentation.
* However, according to FFmpeg documentation, the `-maxrate` and `-bufsiz

In [14]:
from llama_index.core import PromptTemplate
query_engine_rerank = index.as_query_engine(response_mode='refine',similarity_top_k=50, temperature=0.6,node_postprocessors=[reranker_model])

template = (
    "You are hellpful, respectful and honest video transcode assistant and very faimilay with ffmpge, and expecially good at MA35D AMA(AMD multimidia accelerator) device encode/decode/transcode.\n"
    "Context information from multiple sources is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the information from multiple sources and not prior knowledge\n"
    "please read the above context information carefully. and anwer the question.\n"
    "if the question is not releate with video process, just say it is not releated with my knowledge base.\n"
    "if you don't know the answer, just say that I don't know.\n"
    "Answers need to be precise and concise.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_template = PromptTemplate(template)


query_engine_rerank.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)


template = (
    "The original query is as follows: {query_str}.\n"
    "We have provided an existing answer: {existing_answer}.\n"
    "We have the opportunity to refine the existing answer (only if needed) with some more context below.\n"
    "-------------\n"
    "{context_msg}\n"
    "-------------\n"
    "Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\n"
    "if the question is 'who are you' , just say I am a video expert.\n"
    "Answers need to be precise and concise.\n"
    "Refined Answer: "
)


qa_template = PromptTemplate(template)

query_engine_rerank.update_prompts(
    {"response_synthesizer:refine_template": qa_template}
)

prompts_dict = query_engine_rerank.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

You are hellpful, respectful and honest video transcode assistant and very faimilay with ffmpge, and expecially good at MA35D AMA(AMD multimidia accelerator) device encode/decode/transcode.
Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge
please read the above context information carefully. and anwer the question.
if the question is not releate with video process, just say it is not releated with my knowledge base.
if you don't know the answer, just say that I don't know.
Answers need to be precise and concise.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}.
We have provided an existing answer: {existing_answer}.
We have the opportunity to refine the existing answer (only if needed) with some more context below.
-------------
{context_msg}
-------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
if the question is 'who are you' , just say I am a video expert.
Answers need to be precise and concise.
Refined Answer: 


<br><br>

In [15]:
# questions = [
# ]

counter = 0
for question in questions:
   counter = counter + 1
   query_response = query_engine_rerank.query(question)
   print(f"Question{counter}: {question}")
   print(f"Answer:{query_response.response}")

   print("")
   # print(f"source_nodes length:{len(query_response.source_nodes)}")
   # for i, result in enumerate(query_response.source_nodes, start=1):
   #    print(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
05/04/2024 12:09:50 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
05/04/2024 12:09:57 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
05/04/2024 12:10:04 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
05/04/2024 12:10:12 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
05/04/2024 12:10:20 - [INFO] -httpx->>>    HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Question1: explain following ffmpeg command "ffmpeg -hwaccel ama -f rawvideo -s 1920x1080 -framerate 24 -i cut1_1080p.nv12 -vf 'hwupload' -c:v av1_ama -b:v 5M -f mp4 -y 1.av1_1080p_1.mp4" 
Answer:Based on the provided context information, I can refine the given FFmpeg command:

The command is used to transcode a RAW video input file `cut1_10:80p.nv12` into an MP4 container with AV1 codec using AMD MultiMedia Accelerator (AMA) hardware acceleration.

Here's a breakdown of the flags:

* `-hwaccel ama`: This flag enables the AMD MultiMedia Accelerator (AMA) hardware acceleration for the FFmpeg pipeline.
* `-f rawvideo`: The input file format is set to RAW video, which means that the input file does not contain any container or metadata information.
* `-s 19:20x10:80`: The resolution of the input file is specified as 19:20x10:80 (10:80p).
* `-framerate 24`: The target frame rate for the input file is set to 24 frames per second.
* `-i cut1_10:80p.nv12`: This flag specifies the input file n

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/xilinx/Documents/llamaindex_ma35_rag/.venv/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/home/xilinx/Documents/llamaindex_ma35_rag/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/xilinx/Documents/llamaindex_ma35_rag/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/xilinx/Documents/llamaindex_ma35_rag/.venv/lib/python3.10/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py", line 1208, in forward
    outputs = self.roberta(
  File "/home/xilinx/Documents/llamaindex_ma35_rag/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_c

In [ ]:
from llama_index.core import PromptTemplate

# response_mode 有几种模式可选，refine， compact, tree_summarize 等，每一种都有对应的promopt template
query_engine_tree_summarize = index.as_query_engine(response_mode='tree_summarize',similary_threshold=0.1, similarity_top_k=30, node_postprocessors=[reranker_model])

template = (
    "You are a Video ffmpeg & gstreamer technolodge expert and expecially good at MA35D AMA(AMD multimidia accelerator) device encode/decode/scale/transcode.\n"
    "Context information from multiple sources is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the information from multiple sources and not prior knowledge, please read the sources carefully.\n"
    "if the question is not releate with the RDMA, just say it is not releated with my knowledge base.\n"
    "if you don't know the answer, just say that I don't know.\n"
    "if the question is 'who are you' , just say I am a FPGA and RDMA expert.\n"
    "Answers need to be precise and concise.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_template = PromptTemplate(template)

query_engine_tree_summarize.update_prompts(
    {"response_synthesizer:summary_template": qa_template}
)

prompts_dict = query_engine_tree_summarize.get_prompts()
display_prompt_dict(prompts_dict)

In [ ]:
# questions = [
#    "一个ma35D AV1 codec 能处理1080p的数据流最大到多少fps ",
#     """for MA35D video transcode, how to tuning Latency,  for example enable ultra low latency for ama_av1 Encoding?""",
#    """does AMA SDK FFmpeg implement `max_bitrate`, Is max_bitrate implemented in ma35d sdk ffmpeg, and is it correct that the max bitrate parameter in AMA is irrelevant to VBR?""",
# ]

counter = 0
for question in questions:
   counter = counter + 1
   query_response = query_engine_tree_summarize.query(question)
   print(f"Question{counter}: {question}")
   print(f"Answer:{query_response.response}")

   print("")

In [ ]:
from llama_index.core import PromptTemplate
query_engine_refine = index.as_query_engine(response_mode='refine', similarity_top_k=20)

template = (
    "You are video transcode expert and very faimilay with ffmpge and expecially good at MA35D AMA(AMD multimidia accelerator) device encode/decode/scale/transcode.\.\n"
    "Context information from multiple sources is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the information from multiple sources and not prior knowledge\n"
    "please read the above context information carefully. and anwer the question.\n"
    "if the question is not releate with video process, just say it is not releated with my knowledge base.\n"
    "if you don't know the answer, just say that I don't know.\n"
    "Answers need to be precise and concise.\n"
    "if the question is in chinese, please transclate chinese to english in advance"
    "Query: {query_str}\n"
    "Answer: "
)
qa_template = PromptTemplate(template)


query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)


template = (
    "The original query is as follows: {query_str}.\n"
    "We have provided an existing answer: {existing_answer}.\n"
    "We have the opportunity to refine the existing answer (only if needed) with some more context below.\n"
    "-------------\n"
    "{context_msg}\n"
    "-------------\n"
    "Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.\n"
    "if the question is 'who are you' , just say I am a video expert.\n"
    "Answers need to be precise and concise.\n"
    "Refined Answer: "
)


qa_template = PromptTemplate(template)

query_engine.update_prompts(
    {"response_synthesizer:refine_template": qa_template}
)

prompts_dict = query_engine_refine.get_prompts()
display_prompt_dict(prompts_dict)

In [ ]:
# questions = [
#    """Using ffmpeg Decoder a clip that is already encoded in H.264, and will decode the file into a RAW format and save it to disk.""",
#    """Using ffmpeg encode a RAW 1080p60 clip in YUV420 format. Pass the clip to the MA35D encoder to produce an AV1 encoded MP4 output with a target bitrate of 5Mbps and saves it to disk. please do not use -re option""",
#    """ Using ffmpeg do the Bit Conversion, To encode YUV 4:2:2 10 bit pixel format to YUV 4:2:0 8 bit format' """,
#    """ Using ffmpeg decodes an existing H.264 file and then scales it into 1080p/720p/480p/240p four resolutions and save the RAW outputs to disk under""",
#    """ Using ffmpeg one cmd line, decodes an existing H.264 file and then using scaler_ama scales it into 1080p/720p/480p/240p four resolutions and save the RAW outputs to disk under""",
#    # """ffmpeg命令使用ma35d硬件转码, 用一条命令行使用split方式，将一个h264 4k60的文件同时转码成两个hevc和av1格式的文件,写出具体的命令行例子"""
# ]

counter = 0
for question in questions:
   counter = counter + 1
   # query_engine = index.as_query_engine()
   query_response = query_engine_refine.query(question)
   print(f"Question{counter}: {question}")
   print(f"Answer:{query_response.response}")


In [ ]:
# questions = [

# ]

# counter = 0
# for question in questions:
#    counter = counter + 1
#    query_engine = index.as_query_engine()
#    query_response = query_engine.query(question)
#    print(f"Question{counter}: {question}")
#    print(f"Answer:{query_response.response}")